In [20]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model, Sequential
from keras.layers import *
import keras.backend as K
import keras.callbacks

from keras.datasets import mnist

In [21]:
def load_mnist_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    x_train = x_train.reshape(60000, 784)
    return (x_train, y_train, x_test, y_test)

# 1D

In [45]:
def build_1D_generator(rand_shape, output_shape):
    
    data_in = Input(shape=rand_shape, name="data_in")
    
    x = Dense(256, kernel_initializer=initializers.RandomNormal(stddev=0.02))(data_in)       
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Dense(512)(x)       
    x = LeakyReLU(alpha=0.2)(x)

    x = Dense(1024)(x)       
    x = LeakyReLU(alpha=0.2)(x)

    out = Dense(output_shape[1], activation='tanh')(x)
    
    generator = Model(inputs=[data_in], outputs=[out])
    generator.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return generator

In [46]:
def build_1D_discriminator(input_shape):
    
    data_in = Input(shape=input_shape, name="data_in")
    
    x = Dense(1024, kernel_initializer=initializers.RandomNormal(stddev=0.02))(data_in)       
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)
    
    x = Dense(512)(x)       
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    x = Dense(256)(x)       
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)

    out = Dense(1, activation='sigmoid')(x)
    
    discriminator = Model(inputs=[data_in], outputs=[out])
    discriminator.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return discriminator

# 2D

In [47]:
def build_2D_generator(rand_shape, output_shape):
    pass

In [48]:
def build_2D_discriminator(input_shape):
    
    data_in = Input(shape=input_shape, name="data_in")

    x = Convolution2D(filters=64, kernel_size=32, strides=2,
                padding='same', activation="relu", use_bias=False, 
                kernel_initializer=initializers.RandomNormal(stddev=0.02))(data_in)       
    x = BatchNormalization()(x)

    x = Convolution2D(filters=32, kernel_size=16, strides=2, padding='same', 
                      activation="relu", use_bias=False)(x)       
    x = BatchNormalization()(x)

    x = Convolution2D(filters=16, kernel_size=8, strides=2, padding='same', 
                      activation="relu", use_bias=False)(x)       
    x = BatchNormalization()(x)

    x = Flatten(name='flattened')(x)

    out = Dense(1, activation='sigmoid')(x)
    
    discriminator = Model(inputs=[data_in], outputs=[out])
    discriminator.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return discriminator

# Build GAN

In [49]:
def build_gan(generator, discriminator, rand_shape):
    
    # set generation mode
    discriminator.trainable = False
    
    gan_input = Input(shape=rand_shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    
    gan = Model(inputs=[gan_input], outputs=[gan_output])
    gan.compile(loss="binary_crossentropy", optimizer="adadelta")
    
    return gan

# Train GAN

In [52]:
x_train, y_train, x_test, y_test = load_mnist_data()

rand_params = 100
rand_shape = (rand_params, 1)

batch_size = 32
batch_count = x_train.shape[0] / batch_size

output_shape = x_train.shape

generator = build_1D_generator(rand_shape, output_shape)
discriminator = build_1D_discriminator(output_shape)
gan = build_gan(generator, discriminator, rand_shape)

In [ ]:
# TODO Training Process
